In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train_pred=pd.read_csv('oof_df.csv')
test_pred=pd.read_csv('test_pred_df.csv')

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [5]:
train=reduce_mem_usage(train)
train_pred=reduce_mem_usage(train_pred)
test=reduce_mem_usage(test)
test_pred=reduce_mem_usage(test_pred)

Memory usage after optimization is: 606.50 MB
Decreased by 62.5%
Memory usage after optimization is: 20.19 MB
Decreased by 62.5%
Memory usage after optimization is: 134.78 MB
Decreased by 50.0%
Memory usage after optimization is: 16.69 MB
Decreased by 62.5%


In [7]:
train_1=pd.merge(train, train_pred, how='left', left_on='node1_id', right_on='node_id')

In [11]:
train_1=train_1.drop(['is_chat_x','node_id'],axis=1)

In [13]:
train_2=pd.merge(train_1, train_pred, how='left', left_on='node2_id', right_on='node_id')

In [14]:
train_2=train_2.drop(['node_id'],axis=1)

In [16]:
train_2.columns=['node1_id' ,'node2_id','is_chat_x' ,'is_chat_y']

In [18]:
train_2=train_2.fillna(0)

In [19]:
train_2['mean_pred']=(train_2['is_chat_x']+train_2['is_chat_y'])/2

In [22]:
train_2.head()

,node1_id,node2_id,is_chat_x,is_chat_y,mean_pred
0,8446602,6636127,0.082642,0.000000,0.041321
1,1430102,7433949,0.278809,0.000000,0.139404
2,2803017,8372333,0.214233,0.170776,0.192505
3,4529348,894645,0.816895,0.240601,0.528809
4,5096572,4211638,0.900879,0.192749,0.546875


In [23]:
train_2['is_chat']=train['is_chat']

In [24]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [25]:
roc_auc_score(train['is_chat'].values, train_2['mean_pred'].values)

/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


0.7586906698616834

In [26]:
test_1=pd.merge(test, test_pred, how='left', left_on='node1_id', right_on='node_id')

In [28]:
test_1=test_1.drop(['node_id'],axis=1)

In [29]:
test_2=pd.merge(test_1, test_pred, how='left', left_on='node2_id', right_on='node_id')

In [31]:
test_2=test_2.drop(['node_id'],axis=1)

In [35]:
test_2=test_2.fillna(0)

In [36]:
test_2['is_chat']=(test_2['is_chat_x']+test_2['is_chat_y'])/2

In [39]:
test_req=test_2[['id','is_chat']].reset_index(drop=True)

In [43]:
test_req.to_csv('lgb_baseline_pred.csv',index=False)

In [45]:
!zip baseline_preds.zip lgb_baseline_pred.csv

  adding: lgb_baseline_pred.csv (deflated 69%)
